**Calculating the Panel Area of the Ground Truth**
- The calculation is performed by loading information about the target tiles selected during the image selection stage.
- This code is independent of model-specific metrics such as IoU, and is instead used to compute the actual panel area for downstream evaluation or analysis.

In [3]:
import geopandas as gpd
import pandas as pd

# Load GeoPackage
gdf_prediction = gpd.read_file("/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/0.Image_files_selection/final_annotations_PV_all_types_balanced_3_cleaned.gpkg")

# Define masks for each category
is_PV_normal = gdf_prediction['PV_normal'] == 1
is_PV_heater = gdf_prediction['PV_heater'] == 1
is_PV_pool   = gdf_prediction['PV_pool'] == 1
is_uncert    = gdf_prediction['uncertflag'] == 1

# Use 'size' instead of 'area' for summation
pv_normal_by_image = gdf_prediction[is_PV_normal].groupby('image_name')['area'].sum().rename('PV_normal')
pv_heater_by_image = gdf_prediction[is_PV_heater].groupby('image_name')['area'].sum().rename('PV_heater')
pv_pool_by_image   = gdf_prediction[is_PV_pool].groupby('image_name')['area'].sum().rename('PV_pool')
uncert_by_image    = gdf_prediction[is_uncert].groupby('image_name')['area'].sum().rename('Uncertain')

# Merge into a single DataFrame
area_by_image = (
    pd.concat([pv_normal_by_image, pv_heater_by_image, pv_pool_by_image, uncert_by_image], axis=1)
    .fillna(0)
)

# Add a total size column
area_by_image['Total_Predicted_PV_Size'] = area_by_image.sum(axis=1)

# Display result
print(area_by_image)


                        PV_normal   PV_heater      PV_pool  Uncertain  \
image_name                                                              
2023_RGB_8cm_W24A_17  1193.445403  320.143529  1690.707289  45.892695   
2023_RGB_8cm_W25C_16   413.130514  441.588789   324.781132  22.732718   
2023_RGB_8cm_W57B_8   2833.921720  150.003901   995.344031   7.956572   

                      Total_Predicted_PV_Size  
image_name                                     
2023_RGB_8cm_W24A_17              3250.188916  
2023_RGB_8cm_W25C_16              1202.233152  
2023_RGB_8cm_W57B_8               3987.226224  
